# Stops mining 
Detecting stops that are causing issues in the STIB network

In [48]:
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
from tqdm import tqdm
import pandas as pd
import folium
import json
import ast

pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

In [49]:
res = pd.read_pickle("data/computed/matches_EWT_delay.pkl")

In [15]:
stops_id_u = list(res.stop_id.unique())
ids = [i for i in range(len(stops_id_u))]
k = {}
for t in list(zip(ids, stops_id_u)):
    k[t[1]] = t[0]

with open("data/computed/stops_ids.json", "w") as outfile:
    json.dump(k, outfile)

In [52]:
# Late trips
late = res[res.delay_label == "lat"][["route_short_name", "trip_id", "theoretical_time", "stop_sequence" ,"date_normalized", "adj_real_time", "service_id", "th_time_sec", "stop_name", "stop_id", "delay", "delay_label", "vehicule_id"]]
late_g = late.groupby("route_short_name")

In [54]:
for name_l, group_l in tqdm(late_g):
    with open(f"data/line_sequences/seq.data.{name_l}.txt", "w") as data:
        for name, group in group_l.groupby("vehicule_id"):
            group = group.sort_values("stop_sequence").reset_index(drop = True)
            group["label"] = ((group["stop_sequence"] - group.stop_sequence.shift(1)) > 1)
            l = [0] + list(group["label"][group["label"] == True].index) + [group.shape[0]]
            boundaries = [(i, j) for i,j in list(zip(l, list(np.roll(l, -1)))) if j>i]
            vehicle_id = group.iloc[0].vehicule_id
            dt_entry = [(str(vehicle_id), str(i), str(len(group.iloc[j[0]:j[1]].stop_id.to_list())), *group.iloc[j[0]:j[1]].stop_id.to_list()) for i, j in enumerate(boundaries)]
            for ent in dt_entry:
                data.write(" ".join(ent))
                data.write('\n')




100%|██████████| 74/74 [02:43<00:00,  2.21s/it]


In [10]:
spd = []
i = 0

with open("data/computed/seq.data_sub.txt", "w") as data:
    for name, group in tqdm(late_g):
        if i <300:
            group = group.sort_values("stop_sequence").reset_index(drop = True)
            group["label"] = ((group["stop_sequence"] - group.stop_sequence.shift(1)) > 1)
            l = [0] + list(group["label"][group["label"] == True].index) + [group.shape[0]]
            boundaries = [(i, j) for i,j in list(zip(l, list(np.roll(l, -1)))) if j>i]
            vehicle_id = group.iloc[0].vehicule_id
            dt_entry = [(str(vehicle_id), str(i), str(len(group.iloc[j[0]:j[1]].stop_id.to_list())), *group.iloc[j[0]:j[1]].stop_id.to_list()) for i, j in enumerate(boundaries)]
            for ent in dt_entry:
                data.write(" ".join(ent))
                data.write('\n')
        else :
            break
        i += 1


  0%|          | 300/188743 [00:00<04:21, 721.81it/s]


In [22]:
with open("data/computed/seq.PS.data.txt", "w") as dt:
    with open("data/computed/seq.data.txt", "r") as data:
        dta = [row.strip().split(" ")[3:] for row in data.readlines()]
        for ent in dta:
            dt.write(" ".join(ent))
            dt.write('\n')



## Assess Sample

Generic vehicule

In [94]:
sample = late_g.get_group((110345))

In [100]:
sample = sample.sort_values("stop_sequence").reset_index(drop = True)
sample["seq_before"] = sample.stop_sequence.shift(1)
sample["label"] = ((sample["stop_sequence"] - sample["seq_before"]) > 1)
l = [0] + list(sample["label"][sample["label"] == True].index) + [sample.shape[0]]
boundaries = [(i, j) for i,j in list(zip(l, list(np.roll(l, -1)))) if j>i]
vehicle_id = sample.iloc[0].vehicule_id
[(vehicle_id, i, len(sample.iloc[j[0]:j[1]].stop_id.to_list()), sample.iloc[j[0]:j[1]].stop_id.to_list()) for i, j in enumerate(boundaries)]

[(110345, 0, 1, ['8272']),
 (110345, 1, 7, ['8022', '8032', '8042', '8052', '8062', '8072', '8202']),
 (110345, 2, 5, ['8222', '8232', '8242', '8252', '8262'])]

In [96]:
sample

,trip_id,theoretical_time,stop_sequence,date_normalized,adj_real_time,service_id,th_time_sec,stop_name,stop_id,delay,delay_label,vehicule_id,seq_before,label
0,112949572236270000,23:26:55,14,06-09-2021,23:27:44,236270000,84415,SAINTE-CATHERINE,8272,49,lat,110345,NaN,False
1,112949572236270000,23:29:22,16,06-09-2021,23:30:19,236270000,84562,GARE CENTRALE,8022,57,lat,110345,14.0,True
2,112949572236270000,23:30:19,17,06-09-2021,23:31:20,236270000,84619,PARC,8032,61,lat,110345,16.0,False
3,112949572236270000,23:31:25,18,06-09-2021,23:32:52,236270000,84685,ARTS-LOI,8042,87,lat,110345,17.0,False
4,112949572236270000,23:32:40,19,06-09-2021,23:33:23,236270000,84760,MAELBEEK,8052,43,lat,110345,18.0,False
5,112949572236270000,23:33:40,20,06-09-2021,23:34:57,236270000,84820,SCHUMAN,8062,77,lat,110345,19.0,False
6,112949572236270000,23:35:27,21,06-09-2021,23:37:34,236270000,84927,MERODE,8072,127,lat,110345,20.0,False
7,112949572236270000,23:36:57,22,06-09-2021,23:38:05,236270000,85017,THIEFFRY,8202,68,lat,110345,21.0,False
8,112949572236270000,23:39:32,24,06-09-2021,23:41:13,236270000,85172,HANKAR,8222,101,lat,110345,22.0,True
9,112949572236270000,23:40:23,25,06-09-2021,23:41:43,236270000,85223,DELTA,8232,80,lat,110345,24.0,False


In [72]:
l = [0] + list(sample["label"][sample["label"] == True].index) + [sample.shape[0]]
boundaries = [(i, j) for i,j in list(zip(l, list(np.roll(l, -1)))) if j>i]

[(0, 1), (1, 8), (8, 13)]

In [88]:
[(i, sample.iloc[j[0]:j[1]].stop_id.to_list()) for i, j in enumerate(boundaries)]

[(0, ['8272']),
 (1, ['8022', '8032', '8042', '8052', '8062', '8072', '8202']),
 (2, ['8222', '8232', '8242', '8252', '8262'])]

Whole line is late

In [106]:
sample_2 = late_g.get_group((1))

In [107]:
sample_2 = sample_2.sort_values("stop_sequence").reset_index(drop = True)
sample_2

,trip_id,theoretical_time,stop_sequence,date_normalized,adj_real_time,service_id,th_time_sec,stop_name,stop_id,delay,delay_label,vehicule_id
0,112947202236269500,07:30:56,1,18-09-2021,07:40:54,236269500,27056,GARE DE L'OUEST,8733,598,lat,1
1,112947202236269500,07:31:53,2,18-09-2021,07:41:59,236269500,27113,BEEKKANT,8742,606,lat,1
2,112947202236269500,07:33:33,3,18-09-2021,07:43:46,236269500,27213,ETANGS NOIRS,8292,613,lat,1
3,112947202236269500,07:34:44,4,18-09-2021,07:45:02,236269500,27284,COMTE DE FLANDRE,8282,618,lat,1
4,112947202236269500,07:36:08,5,18-09-2021,07:46:21,236269500,27368,SAINTE-CATHERINE,8272,613,lat,1
5,112947202236269500,07:37:12,6,18-09-2021,07:47:11,236269500,27432,DE BROUCKERE,8012,599,lat,1
6,112947202236269500,07:38:28,7,18-09-2021,07:48:27,236269500,27508,GARE CENTRALE,8022,599,lat,1
7,112947202236269500,07:39:23,8,18-09-2021,07:49:06,236269500,27563,PARC,8032,583,lat,1
8,112947202236269500,07:40:29,9,18-09-2021,07:50:07,236269500,27629,ARTS-LOI,8042,578,lat,1
9,112947202236269500,07:41:43,10,18-09-2021,07:51:23,236269500,27703,MAELBEEK,8052,580,lat,1


In [111]:
sample_2["seq_before"] = sample_2.stop_sequence.shift(1)
sample_2["label"] = ((sample_2["stop_sequence"] - sample["seq_before"]) > 1)
l = [0] + list(sample_2["label"][sample_2["label"] == True].index) + [sample_2.shape[0]]
boundaries = [(i, j) for i,j in list(zip(l, list(np.roll(l, -1)))) if j>i]
vehicle_id = sample_2.iloc[0].vehicule_id
[(vehicle_id, i, len(sample_2.iloc[j[0]:j[1]].stop_id.to_list()), *sample_2.iloc[j[0]:j[1]].stop_id.to_list()) for i, j in enumerate(boundaries)]

[(1,
  0,
  16,
  '8733',
  '8742',
  '8292',
  '8282',
  '8272',
  '8012',
  '8022',
  '8032',
  '8042',
  '8052',
  '8062',
  '8072',
  '8082',
  '8092',
  '8102',
  '8112')]

In [119]:
sample_2

,trip_id,theoretical_time,stop_sequence,date_normalized,adj_real_time,service_id,th_time_sec,stop_name,stop_id,delay,delay_label,vehicule_id,seq_before,label
0,112947202236269500,07:30:56,1,18-09-2021,07:40:54,236269500,27056,GARE DE L'OUEST,8733,598,lat,1,NaN,False
1,112947202236269500,07:31:53,2,18-09-2021,07:41:59,236269500,27113,BEEKKANT,8742,606,lat,1,1.0,False
2,112947202236269500,07:33:33,3,18-09-2021,07:43:46,236269500,27213,ETANGS NOIRS,8292,613,lat,1,2.0,False
3,112947202236269500,07:34:44,4,18-09-2021,07:45:02,236269500,27284,COMTE DE FLANDRE,8282,618,lat,1,3.0,False
4,112947202236269500,07:36:08,5,18-09-2021,07:46:21,236269500,27368,SAINTE-CATHERINE,8272,613,lat,1,4.0,False
5,112947202236269500,07:37:12,6,18-09-2021,07:47:11,236269500,27432,DE BROUCKERE,8012,599,lat,1,5.0,False
6,112947202236269500,07:38:28,7,18-09-2021,07:48:27,236269500,27508,GARE CENTRALE,8022,599,lat,1,6.0,False
7,112947202236269500,07:39:23,8,18-09-2021,07:49:06,236269500,27563,PARC,8032,583,lat,1,7.0,False
8,112947202236269500,07:40:29,9,18-09-2021,07:50:07,236269500,27629,ARTS-LOI,8042,578,lat,1,8.0,False
9,112947202236269500,07:41:43,10,18-09-2021,07:51:23,236269500,27703,MAELBEEK,8052,580,lat,1,9.0,False


In [120]:
sample

,trip_id,theoretical_time,stop_sequence,date_normalized,adj_real_time,service_id,th_time_sec,stop_name,stop_id,delay,delay_label,vehicule_id,seq_before,label
0,112949572236270000,23:26:55,14,06-09-2021,23:27:44,236270000,84415,SAINTE-CATHERINE,8272,49,lat,110345,NaN,False
1,112949572236270000,23:29:22,16,06-09-2021,23:30:19,236270000,84562,GARE CENTRALE,8022,57,lat,110345,14.0,True
2,112949572236270000,23:30:19,17,06-09-2021,23:31:20,236270000,84619,PARC,8032,61,lat,110345,16.0,False
3,112949572236270000,23:31:25,18,06-09-2021,23:32:52,236270000,84685,ARTS-LOI,8042,87,lat,110345,17.0,False
4,112949572236270000,23:32:40,19,06-09-2021,23:33:23,236270000,84760,MAELBEEK,8052,43,lat,110345,18.0,False
5,112949572236270000,23:33:40,20,06-09-2021,23:34:57,236270000,84820,SCHUMAN,8062,77,lat,110345,19.0,False
6,112949572236270000,23:35:27,21,06-09-2021,23:37:34,236270000,84927,MERODE,8072,127,lat,110345,20.0,False
7,112949572236270000,23:36:57,22,06-09-2021,23:38:05,236270000,85017,THIEFFRY,8202,68,lat,110345,21.0,False
8,112949572236270000,23:39:32,24,06-09-2021,23:41:13,236270000,85172,HANKAR,8222,101,lat,110345,22.0,True
9,112949572236270000,23:40:23,25,06-09-2021,23:41:43,236270000,85223,DELTA,8232,80,lat,110345,24.0,False


## Evaluate results

In [19]:
stops_sh = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")
lines_sh = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")
seq = pd.read_csv("data/seq/seq.csv")

In [21]:
lines_sh

,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,geometry
0,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
1,001m,2,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (156746.700 170167.000 0.000, 156..."
2,002m,1,#F57000,01/09/2021,06/03/2022,"LINESTRING Z (147305.500 172526.900 0.000, 147..."
3,002m,2,#F57000,01/09/2021,06/03/2022,"LINESTRING Z (147370.500 172498.500 0.000, 147..."
4,003t,1,#B5BA05,01/09/2021,06/03/2022,"LINESTRING Z (148550.000 176641.300 0.000, 148..."
...,...,...,...,...,...,...
169,213b,2,#991F36,01/09/2021,06/03/2022,"LINESTRING Z (144054.200 169925.500 0.000, 144..."
170,216b,1,#80C29C,01/09/2021,06/03/2022,"LINESTRING Z (149229.000 170664.500 0.000, 149..."
171,216b,2,#80C29C,01/09/2021,06/03/2022,"LINESTRING Z (144443.400 173553.200 0.000, 144..."
172,218b,1,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (149173.500 170683.400 0.000, 149..."


In [28]:
stops_lines = stops_sh.merge(
    lines_sh,
    "left",
    left_on = ["Code_Ligne", "Variante"],
    right_on = ["LIGNE", "VARIANTE"],
)

In [13]:
stops = [x for i in seq.sequence for t in ast.literal_eval(i) for x in t]

In [42]:
lines_issues = lines_sh[lines_sh.LIGNE.isin(stops_lines[stops_lines.stop_id.isin(stops)].Code_Ligne)]

In [27]:
seq_issue = stops_sh[stops_sh.stop_id.isin(stops)]

In [47]:
map = folium.Map(location= [50.85045, 4.34878],  zoom_start=12, tiles="cartodbpositron",)

lines_issues.explore(
    m=map,
    color = "LIGNE"
    )

seq_issue.explore(
    m = map,
    color = "red"
    )

